In [38]:
import tensorflow as tf

import pandas as pd

import json
import os
from datetime import date,datetime,timedelta,timezone

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest

print(tf.__version__)

2.12.0


In [39]:
#import functions_framework
# @functions_framework.http
# def predict_incident_severity_by_tf(request):

In [40]:
PATH_FOLDER_ARTIFACTS="model" 
#PATH_FOLDER_ARTIFACTS="tuned_model" 
#PATH_FOLDER_ARTIFACTS="gs://demo-tf-incident-pongthorn/model"

model_version='model_v2_t150723'

PROJECT_ID='pongthorn' # 'smart-data-ml'
dataset_id='SMartML'

client = bigquery.Client(PROJECT_ID)

#credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
#client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

predict_from_date=os.environ.get('predict_from_date', '')
all_prediction=os.environ.get('all_prediction', '1')  # 1 is all , 0 is 1 day


print(f"Prediction From = {predict_from_date}")
print(f"All prediction = {all_prediction}")

# map_sevirity_to_class={'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}

Prediction From = 
All prediction = 1


In [41]:
table_id = f"{PROJECT_ID}.{dataset_id}.new_incident"
predictResult_table_id=f"{PROJECT_ID}.{dataset_id}.new_result_prediction_incident"
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [42]:
print("Load label target multiclasses")
mapping_file="incident_severity_to_class.json"
with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)

print(map_sevirity_to_class)

Load label target multiclasses
{'Cosmetic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [43]:
# Get today's date
prediction_datetime=datetime.now(timezone.utc)
today_str=prediction_datetime.strftime("%Y-%m-%d")
today=datetime.strptime(today_str,"%Y-%m-%d")
print(prediction_datetime)

print(f"Prediction at {prediction_datetime} for {today_str} ({today})")

2023-10-30 15:33:19.367014+00:00
Prediction at 2023-10-30 15:33:19.367014+00:00 for 2023-10-30 (2023-10-30 00:00:00)


In [44]:
def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 return df

In [45]:
if int(all_prediction)==0:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     WHERE DATE(imported_at) = '{today_str}' 
     order by imported_at
    """
else:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     order by imported_at
    """
print(sql)


    SELECT *  FROM `pongthorn.SMartML.new_incident` 
     order by imported_at
    


In [46]:
dfNewData=load_data_bq(sql)

if dfNewData.empty==True:
    print("No Data To predict")
    quit()
    # return "No Data To predict"
else:
    dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='last')
    dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)
    print(dfNewData)
    print(dfNewData.info())


      id  severity_id  severity severity_name                        sla  \
0   3745            2         2         Major    24x7 4Hrs Response Time   
1   3837            2         2         Major    24x7 4Hrs Response Time   
2   3838            2         2         Major    24x7 4Hrs Response Time   
3   3846            2         2         Major  24x7 4Hrs Resolution Time   
4   3847            2         2         Major  24x7 4Hrs Resolution Time   
5   3848            4         0      Cosmetic  24x7 4Hrs Resolution Time   
6   3844            4         0      Cosmetic    24x7 4Hrs Response Time   
7   3858            2         2         Major  24x7 4Hrs Resolution Time   
8   3854            3         1         Minor    24x7 4Hrs Response Time   
9   3852            3         1         Minor    24x7 4Hrs Response Time   
10  3855            4         0      Cosmetic    24x7 6Hrs Response Time   

   product_type      brand service_type         incident_type  \
0       Storage       

In [47]:
try:
    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # print(model.summary())
except Exception as error:
    
  print(str(error))
  raise error


Load from model


In [48]:
pdPrediction=pd.DataFrame(columns=['_id','predict_severity','prob_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}
      predictionResult = model.predict(input_dict)
      result_str=','.join([ str(prob) for prob in predictionResult[0]])  
      print(result_str)   

      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      
      dictPrediction={'_id':incident_id, 'predict_severity':_class,'prob_severity':result_str} 
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([dictPrediction])] )

      print(f"{prob_pct} %   as {_class}")     
      print("======================================================================================")
            
dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData['predict_severity']=dfPredictData['predict_severity'].astype('int')
dfPredictData=dfPredictData[['id','prob_severity','predict_severity','severity']]
dfPredictData['prediction_item_date']= datetime.strptime(today_str, '%Y-%m-%d')
dfPredictData['prediction_datetime']=prediction_datetime
dfPredictData['model_version']=model_version


3745 - 2(Major)
1/1 [==============================] - 0s 255ms/step
0.0039673964,0.25249773,0.74289894,0.0006359592
[[18.61327  23.86481  38.970554 18.551365]] %   as 2
3837 - 2(Major)
1/1 [==============================] - 0s 26ms/step
0.018443331,0.3881473,0.5803142,0.013095099
[[19.24586  27.854649 33.75629  19.143202]] %   as 2
3838 - 2(Major)
1/1 [==============================] - 0s 26ms/step
0.027710058,0.18937078,0.77057546,0.012343647
[[19.003595 22.337994 39.944603 18.713812]] %   as 2
3846 - 2(Major)
1/1 [==============================] - 0s 33ms/step
0.0006580507,0.03255238,0.96333605,0.0034535702
[[17.686882 18.260084 46.316635 17.736395]] %   as 2
3847 - 2(Major)
1/1 [==============================] - 0s 44ms/step
0.00065789284,0.03251572,0.963375,0.0034514014
[[17.686687 18.259216 46.317936 17.736162]] %   as 2
3848 - 0(Cosmetic)
1/1 [==============================] - 0s 65ms/step
0.009496449,0.16295482,0.82053274,0.007015942
[[18.471926 21.535664 41.566242 18.426163]] 

In [49]:
print(dfPredictData.info())
dfPredictData[['id','prediction_item_date','predict_severity','severity']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   id                    11 non-null     object             
 1   prob_severity         11 non-null     object             
 2   predict_severity      11 non-null     int32              
 3   severity              11 non-null     int64              
 4   prediction_item_date  11 non-null     datetime64[ns]     
 5   prediction_datetime   11 non-null     datetime64[ns, UTC]
 6   model_version         11 non-null     object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int32(1), int64(1), object(3)
memory usage: 700.0+ bytes
None


,id,prediction_item_date,predict_severity,severity
0,3745,2023-10-30,2,2
1,3837,2023-10-30,2,2
2,3838,2023-10-30,2,2
3,3846,2023-10-30,2,2
4,3847,2023-10-30,2,2
5,3848,2023-10-30,2,0
6,3844,2023-10-30,0,0
7,3858,2023-10-30,2,2
8,3854,2023-10-30,2,1
9,3852,2023-10-30,1,1


# Save predictoin resutl to Bigquery

In [50]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prob_severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("predict_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity", "INTEGER", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_item_date", "DATETIME", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED"), 
    bigquery.SchemaField("model_version",  "STRING", mode="REQUIRED")     
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new_result_prediction_incident already exists.


In [51]:
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            dfPredictData, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(dfPredictData), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  11 Imported bigquery successfully


In [ ]:
# return 'All incidents has been predicted completely.'

In [ ]:
# if __name__ == "__main__":
#  result=predict_incident_severity_by_tf(None)
#  print(result)
